In [1]:
!pip install scikit-learn

In [2]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [4]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X_df = wine_quality.data.features 
y_df = wine_quality.data.targets 
  
# metadata 
#print(wine_quality.metadata) 
  
# variable information 
#print(wine_quality.variables) 


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Starting the exploratory data analysis. To perform standard pandas operations, I need to convert X and y (which are the features and targets of the dataset, respectively) into pandas DataFrames. I plan to check columns to understand datatypes, value ranges, find possible missing values, the correlation between features, and create relevant data visualizations.

In [9]:
winedf = pd.concat([X_df, y_df], axis=1)

In [ ]:
#winedf.head()

#winedf.columns

#winedf.isnull().sum()

#winedf.describe()

#Great no null values! So I dont have to worry about dropping any rows and my chances of missing data or data that was inputted wrong is lower.

#These data types seem to be correct considering most of htem are percentages in decimal form float64 for would be the best Dtype for them. I also see the quality is int64 I beleive this column is the quality ratings given to each wine which would be whole numbers.

In [ ]:
wine_quality.data

I was having trouble finding the color column with the code given by the UCI to call the data into python. I believe they made a new datasheet to remove the color column. However, for our assignment I have to hot code a catergorical variable and color is the only catergorical column. After running "wine_quality.data" I was able to find the column under "original" so my next step it to pull out the orginal datset and work on that moving forward. 

In [11]:
original_data = wine_quality.data['original']
original_data.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [ ]:
original_data.dtypes

In [ ]:
#time for wine colored graphs

plt.scatter(original_data['quality'], original_data['pH'], color = 'maroon')
plt.title('relation of pH with wine')
plt.xlabel('quality')
plt.ylabel('pH')
plt.legend()
plt.show()

In [ ]:
plt.title('Fixed Acidity vs Quality')
plt.xlabel('Fixed Acidity')
plt.ylabel('Quality')
plt.scatter(original_data['fixed_acidity'], original_data['quality'], color = 'magenta')
plt.show()

In [ ]:
plt.bar(original_data['quality'], original_data['alcohol'], color = 'pink')
plt.title('relation of alcohol to wine')
plt.xlabel('quality')
plt.ylabel('alcohol')
plt.legend()
plt.show()

In [13]:
import seaborn as sns

In [15]:
#Spliting data in two like above

X = original_data.drop(columns=['quality'])  # Features
y = original_data['quality']

print(X.columns)

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'color'],
      dtype='object')


In [17]:
X.shape, y.shape

((6497, 12), (6497,))

In [ ]:
fig = plt.figure(figsize = (10,6))
sns.barplot(X, palette = 'Reds')

Ran into a ValueError when trying to create a correlation matrix so I am going to bring back the features list from the updated dataset because that does not include the color and target(quality) columns.

In [ ]:
corr = winedf.corr()
print(correlation_matrix)

The corrrelation matrix is great to know when proceeding but i belive I will be able to absorb the information better if visualize it.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.show()

From the correlation matrix and plot we can see which features have strong correlation and which features do not. I belive this will be inportant in the future when determining which columns are the most revelant. The alcohol content of the wine has a high correlation with the wine quality. This is important to note because wine quality is what we will working closely with. Some features have negative correlation like density. I plan to remove the features with the most correlation to quality to eliminate some noise if I need to.

In [ ]:
sns.pairplot(original_data, hue='color')

Prediction: I'm not seeing any strong linear relationships in my exploratory graphs. That the random forest will have higher accuracy and F1 scores.

Okay now that I have done all of my exploratory analysis I feel confident to move on to the next step. During my exploratory analysis I did not find any missing values, I know the dtypes of all my columns, and I was able to see the correlation beetween categories. Next I will create a sklearn pipeline that includes preprocessing steps such as scaling and one hot encoding of categorical variable. 

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [25]:
x_train, x_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
x_test

In [27]:
num_cols = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
cat_cols = ['color']

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),  # Scale numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)  # One-hot encode categorical features
    ])


In [ ]:
The first model I will train will be Logicregression

In [ ]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor), ('model', LogisticRegression())])

In [ ]:
pipe.fit(x_train, y_train)


# calculating the training and testing accuracies
print("Training accuracy :", pipe.score(x_train, y_train))
print("Testing accuracy :", pipe.score(x_test, y_test))

y_pred = pipe.predict(x_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

From the results of the test above I'm going to proceed with removing some column/features to reduce noise and hopefully help with the overfitting of the model above

In [ ]:
X.columns

In [ ]:
columnsdrop = ['fixed_acidity','residual_sugar', 'total_sulfur_dioxide', 'pH', 'sulphates']
X_df = X.drop(columns=columnsdrop)

# Proceed with the train-test split and model training
x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.3, random_state=42)

In [ ]:
num_cols1 = ['volatile_acidity', 'citric_acid',
       'chlorides', 'free_sulfur_dioxide', 'density','alcohol']
cat_cols = ['color']

preprocessor1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols1),  # Scale numerical features
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)  # One-hot encode categorical features
    ])

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor1), ('model', LogisticRegression())])

pipe.fit(x_train, y_train)



print("Training accuracy :", pipe.score(x_train, y_train))
print("Testing accuracy :", pipe.score(x_test, y_test))

y_pred = pipe.predict(x_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

The results actually seem to be worse. I believe the results are low because the linear relationships between the features and quality are weak as I predicted above. Now I will move on to randomforest.

In [29]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [31]:
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

# Calculate and print the training and testing accuracies
print("Training accuracy :", pipeline.score(x_train, y_train))
print("Testing accuracy :", pipeline.score(x_test, y_test))

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Training accuracy : 1.0
Testing accuracy : 0.6743589743589744
Classification Report:
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         9
           4       0.67      0.12      0.20        69
           5       0.67      0.74      0.71       613
           6       0.67      0.74      0.70       894
           7       0.69      0.54      0.61       315
           8       0.94      0.33      0.48        49
           9       0.00      0.00      0.00         1

    accuracy                           0.67      1950
   macro avg       0.52      0.35      0.39      1950
weighted avg       0.68      0.67      0.66      1950

Confusion Matrix:
[[  0   0   4   5   0   0   0]
 [  1   8  42  18   0   0   0]
 [  0   3 455 148   7   0   0]
 [  0   1 172 666  55   0   0]
 [  0   0   4 140 170   1   0]
 [  0   0   0  19  14  16   0]
 [  0   0   0   1   0   0   0]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Instantly i can see that these results are a lot better than the logicregression. I'm wondering if the results would be even better excluding some feature so theres less potential for overfitting. To figure out how to make this model a little more accurate I will run a 